# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import pandas as pd
import psycopg2
import glob
from src.scripts.sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkify user=postgres password=password")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    print(filepath)
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("../../data/song_data")

first_song = song_files[0]

../../data/song_data


In [6]:
song_df = pd.read_json(first_song, lines=True)
song_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
# rounding off the duration to the nearest second
song_df['duration'] = round(song_df['duration'])

song_data = song_df[['song_id','title','artist_id','year','duration']]
song_data = tuple(song_data.values[0])
song_data

('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.0)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
cur.execute(song_insert_table, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [18]:
artist_data = song_df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
artist_data = tuple(artist_data.values[0])
artist_data

('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [19]:
cur.execute(artist_insert_table, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [22]:
log_files_list = get_files('../../data/log_data')
first_log = log_files_list[0]

log_df = pd.read_json(first_log, lines=True)
log_df.head()

../../data/log_data


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [27]:
log_df = log_df[log_df['page'] == 'NextSong']

In [28]:
ts = pd.to_datetime(log_df['ts'], unit='ms')
ts.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [33]:
time_data = [(round(dt.timestamp()), dt.hour, dt.day, dt.week, dt.month, dt.year,
              dt.day_name()) for dt in ts]
col_names = ('timestamp', 'hour','day','week','month','year','weekday')
time_data

[(1541903637, 2, 11, 45, 11, 2018, 'Sunday'),
 (1541903771, 2, 11, 45, 11, 2018, 'Sunday'),
 (1541904035, 2, 11, 45, 11, 2018, 'Sunday'),
 (1541910974, 4, 11, 45, 11, 2018, 'Sunday'),
 (1541911007, 4, 11, 45, 11, 2018, 'Sunday'),
 (1541930189, 9, 11, 45, 11, 2018, 'Sunday'),
 (1541930448, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541930648, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541930909, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541931121, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541932064, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541932425, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541932583, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541932815, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541933004, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541933179, 10, 11, 45, 11, 2018, 'Sunday'),
 (1541934892, 11, 11, 45, 11, 2018, 'Sunday'),
 (1541943683, 13, 11, 45, 11, 2018, 'Sunday'),
 (1541944344, 13, 11, 45, 11, 2018, 'Sunday'),
 (1541944703, 13, 11, 45, 11, 2018, 'Sunday'),
 (1541945144, 14, 11, 45, 11, 2018, 'Sunday'),
 (1541945275, 14, 1

In [34]:
time_df = pd.DataFrame(time_data,columns=col_names)
time_df

,timestamp,hour,day,week,month,year,weekday
0,1541903637,2,11,45,11,2018,Sunday
1,1541903771,2,11,45,11,2018,Sunday
2,1541904035,2,11,45,11,2018,Sunday
3,1541910974,4,11,45,11,2018,Sunday
4,1541911007,4,11,45,11,2018,Sunday
...,...,...,...,...,...,...,...
68,1541970569,21,11,45,11,2018,Sunday
69,1541970829,21,11,45,11,2018,Sunday
70,1541971050,21,11,45,11,2018,Sunday
71,1541979541,23,11,45,11,2018,Sunday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [35]:
for i, row in time_df.iterrows():
    cur.execute(time_insert_table, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [38]:
user_df = log_df[['userId','firstName','lastName','gender','level']]
user_df = user_df[user_df['userId'].astype(bool)]
user_df = user_df.drop_duplicates(subset='userId')
user_df

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
6,92,Ryann,Smith,F,free
12,49,Chloe,Cuevas,F,free
27,88,Mohammad,Rodriguez,M,free
30,96,Cierra,Finley,F,free
31,26,Ryan,Smith,M,free
35,44,Aleena,Kirby,F,paid
48,67,Colm,Santana,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [39]:
for i, row in user_df.iterrows():
    cur.execute(users_insert_table, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [45]:
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [48]:
# insert songplay records
for i, row in log_df.iterrows():

    song_name = row.song
    artist_name = row.artist
    song_duration = round(row.length)

    cur.execute(song_select, (song_name, artist_name, song_duration))
    result = cur.fetchone()
    (song_id, artist_id) = (result if result else (None, None))

    if song_id is None or artist_id is None:
        continue

    songplay_data = (round(row.ts / 1000.0), row.userId, row.level,
                     song_id, artist_id, row.sessionId, row.location,
                     row.userAgent)

    cur.execute(songplays_insert_table, songplay_data)
    conn.commit()

print('Completed')

Completed


In [50]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.